# One classifier each name: OCEN (Author group)
This method simply ignored the records below threshold

In [1]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import com_func

def dummy(doc):
    return doc

# parameters
threshold = 100
cutoff = 3

pp_textual = ["tf","tf_idf","lsa"]

Dataset = "pubmed"

In [2]:
import pickle
import gensim
# read trained rec to rec textual graph
def read_textual_embedding(Dataset = "pubmed", emb_type = "off"):
    textual_emb = []
    emb_pid = []
    while True:
        if emb_type == "tf":
            modelSaveDir = "../../Data/"+Dataset+"/models/tf/textual_sample=140k/"
            with open(modelSaveDir+'tf_features.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec.toarray()
            emb_pid = allPaperid
            break
        elif emb_type == "tfidf":
            modelSaveDir = "../../Data/"+Dataset+"/models/tf_idf/textual_sample=140k/"
            with open(modelSaveDir+'tf_idf_trained_features.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec.toarray()
            emb_pid = allPaperid
            break
        elif emb_type == "lsa":
            modelSaveDir = "../../Data/"+Dataset+"/models/lsa/textual_sample=140k/"
            with open(modelSaveDir+'lsa_Matrix.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec
            emb_pid = allPaperid
            break
        elif emb_type == "pv_dm":
            modelSaveDir = "../../Data/"+Dataset+"/models/doc2v/textual_sample=140k/"
            model = gensim.models.Doc2Vec.load(modelSaveDir+"pv_dm/Doc2Vec(dmm,d100,n5,w5,mc2,s0.001,t24)")
            allPaperTags = model.docvecs.offset2doctag
            for pid in allPaperTags:
                vectorRepresentation = model.docvecs[pid].tolist()
                vectorRepresentation = [float(i) for i in vectorRepresentation]
                textual_emb.append(vectorRepresentation)
            emb_pid = allPaperTags
            break
        elif emb_type == "pv_dbow":
            modelSaveDir = "../../Data/"+Dataset+"/models/doc2v/textual_sample=140k/"
            model = gensim.models.Doc2Vec.load(modelSaveDir+"pv_dbow/Doc2Vec(dbow,d100,n5,mc2,s0.001,t24)")

            allPaperTags = model.docvecs.offset2doctag
            for pid in allPaperTags:
                vectorRepresentation = model.docvecs[pid].tolist()
                vectorRepresentation = [float(i) for i in vectorRepresentation]
                textual_emb.append(vectorRepresentation)
            emb_pid = allPaperTags
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    print("Total textual vector records:",len(textual_emb))
    print("Vector dimension: ", len(textual_emb[0]))
    return textual_emb, emb_pid

In [3]:
# read trained rec to rec node2vec citation graph
def read_citation_embedding(Dataset = "pubmed", emb_type = "off"):
    citation_emb = []
    emb_pid = []
    while True:
        if emb_type == "n2v":
            citation_emb_dir = "../../Data/"+Dataset+"/vectors/"+emb_type+"/extracted_labeled_n2v.txt"
            with open(citation_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    if(len(read_data)==101):
                        emb_pid.append(read_data[0])
                        citation_emb.append(read_data[1:])
            f.close()
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    print("Total citation vector records:",len(citation_emb))
    print("Vector dimension: ", len(citation_emb[0]))
    return citation_emb, emb_pid

In [4]:
import pandas as pd
def dummy(doc):
    return doc
def read_labeled_file(infile):
    LabeledRecords_original = []
    with open(infile, 'r', encoding = 'utf8') as f:
        for line in f:
            read_data = line.split("\t")
            # get ride of bad formated lines
            if(len(read_data)==13 or len(read_data)==12):
                paper_detail = {"paperID": read_data[0], "authorID":read_data[1], 
                                "co-author": read_data[5], "venue_id": read_data[7]}
                LabeledRecords_original.append(paper_detail)
            else:
                print(len(read_data))
        f.close()
    return pd.DataFrame(LabeledRecords_original)

In [5]:
def extract_embedding(all_embedding, all_embedding_pid, wanted_pid_list):
    extracted_emb = []
    wanted_pid_list = wanted_pid_list.values.tolist()
    # only if embedding exist
    if len(all_embedding)>0:
        # loop through wanted pid list to keep input order
        for wanted_pid in wanted_pid_list:
            # if wanted paper in all pretrained embeddings
            if wanted_pid in all_embedding_pid:
                emb_idx = all_embedding_pid.index(wanted_pid)
                extracted_emb.append(all_embedding[emb_idx])
            # if wanted paper not in all pretrained embeddings, fill missing sample with 0's
            else:
                print("Missing Sample: ", wanted_pid)
                temp = [0] * len(all_embedding[0])
                extracted_emb.append(temp)
                
    extracted_emb = pd.DataFrame(extracted_emb)
    return extracted_emb

In [6]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score
# cross validation
def k_fold_cv(data, label, clf, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for train_index, test_index in kf.split(data):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        # fit data to clf
        clf.fit(data_train, label_train)
        # get predicted label
        label_pred = clf.predict(data_test)
        allTrueLabel.extend(label_test)
        allPredLabel.extend(label_pred)

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='macro')
    
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    
    # accumulate statistic for entire model f1
    cnf_matrix = confusion_matrix(allTrueLabel, allPredLabel)
    TP = np.diag(cnf_matrix)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
#     print(cnf_matrix)
#     print("TP: ",TP, "TN: ",TN, "FP: ",FP,"FN: ",FN)

    return accuracy, f1, TP.sum(), TN.sum(), FP.sum(), FN.sum()

In [7]:
def write_csv_df(savePath, filename, df):
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    # Give the filename you wish to save the file to
    pathfile = os.path.normpath(os.path.join(savePath,filename))

    # Use this function to search for any files which match your filename
    files_present = os.path.isfile(pathfile) 
    # if no matching files, write to csv, if there are matching files, print statement
    if not files_present:
        df.to_csv(pathfile, encoding='utf-8',index=False)
    else:
        overwrite = input("WARNING: " + pathfile + " already exists! Do you want to overwrite <y/n>? \n ")
        if overwrite == 'y':
            df.to_csv(pathfile, encoding='utf-8',index=False)
        elif overwrite == 'n':
            new_filename = input("Type new filename: \n ")
            write_csv_df(savePath,new_filename,df)
        else:
            print("Not a valid input. Data is NOT saved!\n")

In [8]:
# read pretrained embeddings
all_textual_embedding, all_textual_emb_pid = read_textual_embedding(emb_type = "tf")
all_citation_embedding, all_citation_emb_pid = read_citation_embedding(emb_type = "n2v")

Total textual vector records: 135796
Vector dimension:  50000
Total citation vector records: 124922
Vector dimension:  100


In [9]:
print(all_textual_embedding[0])
print(all_textual_emb_pid[0])
print(all_citation_embedding[0])
print(all_citation_emb_pid[0])

[0. 0. 0. ... 0. 0. 0.]
8077
['0.074837', '0.437304', '0.157833', '0.179944', '-0.0696371', '-0.0925071', '-0.37209', '0.16441', '0.257381', '0.482553', '0.420752', '0.294299', '0.48322', '0.310536', '0.451489', '-0.0321524', '-0.266308', '-0.507235', '0.302519', '-0.192578', '-0.196128', '-0.716089', '0.118927', '0.130549', '0.0538411', '-0.36721', '0.320577', '0.107628', '0.437685', '0.261019', '-0.134182', '0.467584', '-0.433934', '-0.337566', '-0.112999', '0.131627', '0.185436', '-0.0716854', '0.222004', '-0.296244', '0.0662622', '0.209887', '-0.177259', '-0.202866', '0.206727', '-0.0535898', '-0.0832955', '0.00406953', '-0.13292', '-0.0853675', '-0.241761', '-0.327425', '-0.46692', '0.0485383', '0.00806723', '0.0284221', '0.115838', '-0.255672', '-0.770949', '0.0873891', '0.00681434', '0.0626846', '-0.0590345', '0.299776', '-0.173271', '-0.00270774', '-0.498401', '-0.222046', '0.321921', '0.0837049', '-0.0501312', '-0.284909', '0.274566', '0.0670506', '0.0773459', '0.24957', '-0.0

In [116]:
# load the file
import io
import collections
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

# entire model f1
modelSVCf1, modelLRf1 = ([] for i in range(2))

# collect statistic to output
allname, num_class, per_class_count = ([] for i in range(3))

all_svcLinear_accuracy, all_svcLinear_f1, all_LR_accuracy, all_LR_f1 = ([] for i in range(4))

# collect overall tp, tn, fp, fn
svcTP=svcTN=svcFP=svcFN = 0
lrTP=lrTN=lrFP=lrFN = 0
# read all file in labeled group
for file in listfiles:
    # group name
    temp = file.split("_")
    name = temp[1]+"_"+temp[-1]
    print("For name: ",name)
    # read needed content in labeled file
    labeled_data = read_labeled_file(fileDir+file)
    print("total sample size before apply threshold: ",len(labeled_data))
    # count number of paper each author write based on author ID
    paperCounter = collections.Counter(labeled_data["authorID"])
    print(paperCounter)
    # collect per class statistic
    for k in list(paperCounter):
        if paperCounter[k] < threshold:
            del paperCounter[k]
    temp =list(paperCounter.keys())
    print(temp)
    # remove samples that are smaller than threshold
    labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
    print("Total sample size after apply threshold: ",len(labeled_data))
    # if only have one class or no class pass the threshold, not applicable
    if(len(paperCounter)==0) or (len(paperCounter)==1):
        print(name, " pass")
    else:
        allname.append(name)
        num_class.append(len(paperCounter))
        per_class_count.append(paperCounter)
        # shuffle the data
        labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
        # extract true label and pid
        label = labeled_data["authorID"]
        pid = labeled_data["paperID"]
        # list of different data field
        part_collection = []
        # select feature wanted to fit to clustering/classification algorithm
        # data part, textual information
        data_part_textual = extract_embedding(all_textual_embedding, all_textual_emb_pid, pid)
        print(data_part_textual.shape)
        part_collection.append(data_part_textual)
        # data part, citation information
        data_part_citation = extract_embedding(all_citation_embedding, all_citation_emb_pid, pid)
        print(data_part_citation.shape)
        part_collection.append(data_part_citation)
        # merge different part of data data together by concatenate it all together
        # remove empty emb (when emb set off)
        part_collection = [part for part in part_collection if len(part)!=0]
        print(len(part_collection))
        if len(part_collection)>1:
            combinedata = np.concatenate(part_collection,axis=1)
        elif len(part_collection)==1:
            if isinstance(part_collection[0], pd.DataFrame):
                combinedata = part_collection[0].values
            else:
                combinedata = part_collection[0]
        else:
            print("No data available")
            break
        print(combinedata.shape)
        # using converted feature vector to train classifier
        # using SVM with linear kernal
        clf = SVC(decision_function_shape='ovr', kernel='linear')
        svcaccuracy, svcmarcof1, tp, tn, fp, fn = k_fold_cv(combinedata, label, clf, k=10)
        svcTP+=tp
        svcTN+=tn
        svcFP+=fp
        svcFN+=fn
        print("svc Accuracy: ",svcaccuracy)
        print("svc F1: ", svcmarcof1)
        all_svcLinear_accuracy.append(svcaccuracy)
        all_svcLinear_f1.append(svcmarcof1)
        # using logistic regression
        clf = LogisticRegression(multi_class='ovr')
        LRaccuracy, LRmarcof1, tp, tn, fp, fn = k_fold_cv(combinedata, label, clf, k=10)
        lrTP+=tp
        lrTN+=tn
        lrFP+=fp
        lrFN+=fn
        print("LR Accuracy: ",LRaccuracy)
        print("LR F1: ", LRmarcof1)
        all_LR_accuracy.append(LRaccuracy)
        all_LR_f1.append(LRmarcof1)
    break
# print f1 for entire model
print("svc: TP: ",svcTP, "TN: ",svcTN, "FP: ",svcFP,"FN: ",svcFN)
print("lr: TP: ",lrTP, "TN: ",lrTN, "FP: ",lrFP,"FN: ",lrFN)
svcF1 = 2*svcTP / (2*svcTP + svcFP + svcFN)
lrF1 = 2*lrTP / (2*lrTP + lrFP + lrFN)
modelSVCf1.append(svcF1)
modelLRf1.append(lrF1)
# # write evaluation result to excel
# output = pd.DataFrame({'Name Group':allname,"Class number":num_class,"per_class_size":per_class_count, 
#                        "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
#                        "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

# savePath = "../../result/"+Dataset+"/skovr/"
# filename = "textual="+emb+"_threshold="+str(threshold)+".csv"
# write_csv_df(savePath, filename, output)
# print("Done")

For name:  j_read
total sample size before apply threshold:  136
Counter({'0000-0002-5159-1192': 57, '0000-0002-9029-5185': 39, '0000-0002-9697-0962': 31, '0000-0002-4739-9245': 3, '0000-0003-0605-5259': 3, '0000-0003-4316-7006': 1, '0000-0002-0784-0091': 1, '0000-0002-3888-6631': 1})
['0000-0002-9697-0962', '0000-0002-9029-5185', '0000-0002-5159-1192']
Total sample size after apply threshold:  127
(127, 50000)
Missing Sample:  20516165
Missing Sample:  12803940
Missing Sample:  22427696
Missing Sample:  17233514
Missing Sample:  16946369
Missing Sample:  23482297
Missing Sample:  25091803
Missing Sample:  27990325
Missing Sample:  26963178
Missing Sample:  24071587
(127, 100)
2
(127, 50100)
                     precision    recall  f1-score   support

0000-0002-5159-1192       0.93      0.98      0.96        57
0000-0002-9029-5185       0.97      0.97      0.97        39
0000-0002-9697-0962       1.00      0.90      0.95        31

          micro avg       0.96      0.96      0.96   

In [32]:
print(pp_textual)
print("svc: ", modelSVCf1)
print("lr: ", modelLRf1)

['tf', 'tf_idf', 'lsa']
svc:  [0.9464991405128569, 0.9731343812578639, 0.9706356660582325]
lr:  [0.9699976962200287, 0.9314359637774903, 0.9415017101135941]


In [19]:
# accuracy
from statistics import mean 
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

578
578
0.9798807689357634
0.9496564529342066


In [20]:
# f1
from statistics import mean 
# remove string from result
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

578
578
0.9764067884953497
0.9288583090243283


In [ ]:
%reset

In [ ]:
%who